In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import librosa.display



In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
music_data = pd.read_csv('drive/My Drive/gtzan_dataset/Data/features_30_sec.csv')
music_data.head(5)

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,blues.00001.wav,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,blues.00002.wav,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,blues.00003.wav,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,blues.00004.wav,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues


In [ ]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
music_data['label'] = label_encoder.fit_transform(music_data['label'])
X = music_data.drop(['label','filename'],axis=1)
y = music_data['label']

cols = X.columns
minmax = preprocessing.MinMaxScaler()
np_scaled = minmax.fit_transform(X)

X = pd.DataFrame(np_scaled, columns = cols)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
													test_size=0.3,
													random_state=111)
X_train.shape, X_test.shape, y_train.shape, y_test.shape



((700, 58), (300, 58), (700,), (300,))

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

xgb_classifier = xgb.XGBClassifier()

xgb_classifier.fit(X_train, y_train)

y_pred_xgb = xgb_classifier.predict(X_test)

accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print("XGBoost Accuracy:", accuracy_xgb)

k = 5
cv_scores = cross_val_score(xgb_classifier, X_train, y_train, cv=k)

mean_accuracy = cv_scores.mean()
print("Mean Cross-Validated Accuracy:", mean_accuracy)


XGBoost Accuracy: 0.7633333333333333
Mean Cross-Validated Accuracy: 0.74


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

rf_classifier.fit(X_train, y_train)

y_pred_rf = rf_classifier.predict(X_test)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Random Forest Accuracy:", accuracy_rf)


from sklearn.model_selection import cross_val_score

k = 5
cv_scores = cross_val_score(rf_classifier, X_train, y_train, cv=k)

mean_accuracy = cv_scores.mean()
print("Mean Cross-Validated Accuracy:", mean_accuracy)


Random Forest Accuracy: 0.78
Mean Cross-Validated Accuracy: 0.7614285714285713


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


knn_classifier = KNeighborsClassifier(n_neighbors=5)

knn_classifier.fit(X_train, y_train)

y_pred_knn = knn_classifier.predict(X_test)

accuracy_knn = accuracy_score(y_test, y_pred_knn)
print("KNN Accuracy:", accuracy_knn)


KNN Accuracy: 0.7033333333333334


In [ ]:
from sklearn.naive_bayes import GaussianNB

nb_classifier = GaussianNB()

nb_classifier.fit(X_train, y_train)

y_pred = nb_classifier.predict(X_test)

accuracy = nb_classifier.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.5633333333333334


In [ ]:
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *

model = Sequential()

model.add(Flatten(input_shape=(58,)))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(10, activation='softmax'))

model.summary()



Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 58)                0         
                                                                 
 dense_9 (Dense)             (None, 256)               15104     
                                                                 
 batch_normalization_3 (Bat  (None, 256)               1024      
 chNormalization)                                                
                                                                 
 dense_10 (Dense)            (None, 128)               32896     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                1290      
                                                      

In [ ]:

adam = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer=adam,
			loss="sparse_categorical_crossentropy",
			metrics=["accuracy"])

hist = model.fit(X_train, y_train,
				validation_data = (X_test,y_test),
				epochs = 100,
				batch_size = 32)
test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test accuracy: {test_accuracy}")



Epoch 1/100
22/22 [==============================] - 1s 14ms/step - loss: 1.8890 - accuracy: 0.3314 - val_loss: 2.1808 - val_accuracy: 0.1867
Epoch 2/100
22/22 [==============================] - 0s 5ms/step - loss: 1.2282 - accuracy: 0.5514 - val_loss: 2.1267 - val_accuracy: 0.0967
Epoch 3/100
22/22 [==============================] - 0s 6ms/step - loss: 1.0215 - accuracy: 0.6400 - val_loss: 2.0541 - val_accuracy: 0.1467
Epoch 4/100
22/22 [==============================] - 0s 5ms/step - loss: 0.8803 - accuracy: 0.7029 - val_loss: 1.9647 - val_accuracy: 0.2900
Epoch 5/100
22/22 [==============================] - 0s 5ms/step - loss: 0.8130 - accuracy: 0.7243 - val_loss: 1.9076 - val_accuracy: 0.3833
Epoch 6/100
22/22 [==============================] - 0s 5ms/step - loss: 0.7011 - accuracy: 0.7643 - val_loss: 1.8163 - val_accuracy: 0.5133
Epoch 7/100
22/22 [==============================] - 0s 5ms/step - loss: 0.6450 - accuracy: 0.8014 - val_loss: 1.7184 - val_accuracy: 0.6433
Epoch 8/100


In [ ]:
music_data = pd.read_csv('drive/My Drive/gtzan_dataset/Data/features_3_sec.csv')
music_data.head(5)


,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.0.wav,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,blues.00000.1.wav,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,blues.00000.2.wav,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,blues.00000.3.wav,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,blues.00000.4.wav,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues


In [ ]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
music_data['label'] = label_encoder.fit_transform(music_data['label'])
X = music_data.drop(['label','filename'],axis=1)
y = music_data['label']

cols = X.columns
minmax = preprocessing.MinMaxScaler()
np_scaled = minmax.fit_transform(X)

X = pd.DataFrame(np_scaled, columns = cols)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
													test_size=0.3,
													random_state=111)
X_train.shape, X_test.shape, y_train.shape, y_test.shape



((6993, 58), (2997, 58), (6993,), (2997,))

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

xgb_classifier = xgb.XGBClassifier()

xgb_classifier.fit(X_train, y_train)

y_pred_xgb = xgb_classifier.predict(X_test)


k = 5
cv_scores = cross_val_score(xgb_classifier, X_train, y_train, cv=k)

mean_accuracy = cv_scores.mean()
print("Mean Cross-Validated Accuracy:", mean_accuracy)


Mean Cross-Validated Accuracy: 0.8778782310274762


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

rf_classifier.fit(X_train, y_train)

y_pred_rf = rf_classifier.predict(X_test)

from sklearn.model_selection import cross_val_score

k = 5
cv_scores = cross_val_score(rf_classifier, X_train, y_train, cv=k)

mean_accuracy = cv_scores.mean()
print("Mean Cross-Validated Accuracy:", mean_accuracy)

Mean Cross-Validated Accuracy: 0.8404118617324248


In [ ]:
from sklearn.naive_bayes import GaussianNB

nb_classifier = GaussianNB()

nb_classifier.fit(X_train, y_train)

y_pred = nb_classifier.predict(X_test)

accuracy = nb_classifier.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.5151818485151819


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


knn_classifier = KNeighborsClassifier(n_neighbors=5)

knn_classifier.fit(X_train, y_train)

y_pred_knn = knn_classifier.predict(X_test)

accuracy_knn = accuracy_score(y_test, y_pred_knn)
print("KNN Accuracy:", accuracy_knn)


KNN Accuracy: 0.8932265598932265


In [ ]:
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *

model = Sequential()

model.add(Flatten(input_shape=(58,)))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(10, activation='softmax'))

model.summary()



Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 58)                0         
                                                                 
 dense_15 (Dense)            (None, 256)               15104     
                                                                 
 batch_normalization_5 (Bat  (None, 256)               1024      
 chNormalization)                                                
                                                                 
 dense_16 (Dense)            (None, 128)               32896     
                                                                 
 dropout_5 (Dropout)         (None, 128)               0         
                                                                 
 dense_17 (Dense)            (None, 10)                1290      
                                                      

In [ ]:

adam = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer=adam,
			loss="sparse_categorical_crossentropy",
			metrics=["accuracy"])

hist = model.fit(X_train, y_train,
				validation_data = (X_test,y_test),
				epochs = 100,
				batch_size = 32)
test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test accuracy: {test_accuracy}")



Epoch 1/100
219/219 [==============================] - 2s 5ms/step - loss: 1.2550 - accuracy: 0.5546 - val_loss: 1.7448 - val_accuracy: 0.4918
Epoch 2/100
219/219 [==============================] - 1s 4ms/step - loss: 0.9600 - accuracy: 0.6601 - val_loss: 1.1211 - val_accuracy: 0.6340
Epoch 3/100
219/219 [==============================] - 1s 4ms/step - loss: 0.8415 - accuracy: 0.7121 - val_loss: 0.8098 - val_accuracy: 0.7201
Epoch 4/100
219/219 [==============================] - 1s 4ms/step - loss: 0.7829 - accuracy: 0.7239 - val_loss: 0.8779 - val_accuracy: 0.6850
Epoch 5/100
219/219 [==============================] - 1s 4ms/step - loss: 0.7330 - accuracy: 0.7402 - val_loss: 0.7655 - val_accuracy: 0.7244
Epoch 6/100
219/219 [==============================] - 1s 4ms/step - loss: 0.6914 - accuracy: 0.7606 - val_loss: 0.6920 - val_accuracy: 0.7541
Epoch 7/100
219/219 [==============================] - 1s 4ms/step - loss: 0.6579 - accuracy: 0.7675 - val_loss: 0.7719 - val_accuracy: 0.7347